# Improving Performance
Improve data cleaning tasks by increasing performance or reducing resource requirements.

## Caching a DataFrame
You've been assigned a task that requires running several analysis operations on a DataFrame. You've learned that caching can improve performance when reusing DataFrames and would like to implement it.

You'll be working with a new dataset consisting of airline departure information. It may have repetitive data and will need to be de-duplicated.

In [2]:
import time

# File Path
file_path = ".../data/datacamp/flights_csv/AA_DFW_2017.csv"

# Load the CSV file
departures_df = spark.read.format('csv').options(Header=True).load(file_path).repartition(12)

start_time = time.time()

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 139358 rows took 28.526904 seconds
Counting 139358 rows again took 10.513554 seconds


Consider why the first run takes longer even though you've told it to cache() the DataFrame. Remember that even though you've applied the caching transformation, it doesn't take effect until an action is run. The action instantiates the caching after the distinct() function completes. The second time, there is no need to recalculate anything so it returns almost immediately.

## Removing a DataFrame from cache
You've finished the analysis tasks with the departures_df DataFrame, but have some other processing to do. You'd like to remove the DataFrame from the cache to prevent any excess memory usage on your cluster.

In [3]:
# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()

# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)

Is departures_df cached?: True
Removing departures_df from cache
Is departures_df cached?: False


## File import performance
You've been given a large set of data to import into a Spark DataFrame. You'd like to test the difference in import speed by splitting up the file.

You have two types of files available: departures_full.txt.gz and departures_xxx.txt.gz where xxx is 000 - 013. The same number of rows is split between each file.

In [4]:
# File Path
file_path = ".../data/datacamp/departures/"

# Import the full and split files into DataFrames
full_df = spark.read.csv(file_path + 'full/')
split_df = spark.read.csv(file_path + 'split/')

# Print the count and run time for each DataFrame
start_time_a = time.time()
print("Total rows in full DataFrame:\t%d" % full_df.count())
print("Time to run: %f" % (time.time() - start_time_a))

start_time_b = time.time()
print("Total rows in split DataFrame:\t%d" % split_df.count())
print("Time to run: %f" % (time.time() - start_time_b))

Total rows in full DataFrame:	139358
Time to run: 0.709241
Total rows in split DataFrame:	259268
Time to run: 2.388764


The results should illustrate that using split files runs more quickly than using one large file for import. Note that in certain circumstances the results may be reversed. This is a side effect of running as a single node cluster. Depending on the tasks required and resources available, it may occasionally take longer than expected. If you perform multiple runs of the tasks, you should see the full file import as generally slower than the split file import.

## Reading Spark configurations
You've recently configured a cluster via a cloud provider. Your only access is via the command shell or your python code. You'd like to verify some Spark settings to validate the configuration of the cluster.

In [5]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: Cleaning Data with PySpark
Driver TCP port: 39601
Number of partitions: 5000


## Writing Spark configurations
Now that you've reviewed some of the Spark configurations on your cluster, you want to modify some of the settings to tune Spark to your needs. You'll import some data to review that your changes have affected the cluster.

The spark configuration is initially set to the default value of 200 partitions.

In [6]:
# Store the number of partitions in variable
before = split_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv(file_path + 'split/').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Partition count before change: 13
Partition count after change: 500


It's important to remember that modifying the settings in Spark may change objects that already exist. Sometimes the changes only take effect after configuring a new DataFrame. Remember to test changes you make to Spark configurations to verify it does exactly what you think.

## Normal joins
You've been given two DataFrames to combine into a single useful DataFrame. Your first task is to combine the DataFrames normally and view the execution plan.

In [7]:
file_path = ".../data/datacamp/"

flights_df = spark.read.csv(file_path + "flights_csv/AA_DFW_2018.csv", header=True).repartition(50)
airports_df = spark.read.parquet(file_path + "airports_joins/")

# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.join(airports_df, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.explain()

== Physical Plan ==
*(3) BroadcastHashJoin [Destination Airport#192], [IATA#199], Inner, BuildRight
:- Exchange RoundRobinPartitioning(50)
:  +- *(1) Project [Date (MM/DD/YYYY)#190, Flight Number#191, Destination Airport#192, Actual elapsed time (Minutes)#193]
:     +- *(1) Filter isnotnull(Destination Airport#192)
:        +- *(1) FileScan csv [Date (MM/DD/YYYY)#190,Flight Number#191,Destination Airport#192,Actual elapsed time (Minutes)#193] Batched: false, Format: CSV, Location: InMemoryFileIndex[s3a://qa.dssa.thetradedesk.com/data/datacamp/flights_csv/AA_DFW_2018.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Destination Airport)], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight Number:string,Destination Airport:string,Actual elapsed ti...
+- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, true]))
   +- *(2) Project [AIRPORTNAME#198, IATA#199]
      +- *(2) Filter isnotnull(IATA#199)
         +- *(2) FileScan parquet [AIRPORTNAME#198,IATA#199] Batched: 

## Using broadcasting on Spark joins
Remember that table joins in Spark are split between the cluster workers. If the data is not local, various shuffle operations are required and can have a negative impact on performance. Instead, we're going to use Spark's broadcast operations to give each node a copy of the specified data.

A couple tips:

- Broadcast the smaller DataFrame. The larger the DataFrame, the more time required to transfer to the worker nodes.
- On small DataFrames, it may be better skip broadcasting and let Spark figure out any optimization on its own.
- If you look at the query execution plan, a broadcastHashJoin indicates you've successfully configured broadcasting.

In [8]:
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.join(broadcast(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

== Physical Plan ==
*(3) BroadcastHashJoin [Destination Airport#192], [IATA#199], Inner, BuildRight
:- Exchange RoundRobinPartitioning(50)
:  +- *(1) Project [Date (MM/DD/YYYY)#190, Flight Number#191, Destination Airport#192, Actual elapsed time (Minutes)#193]
:     +- *(1) Filter isnotnull(Destination Airport#192)
:        +- *(1) FileScan csv [Date (MM/DD/YYYY)#190,Flight Number#191,Destination Airport#192,Actual elapsed time (Minutes)#193] Batched: false, Format: CSV, Location: InMemoryFileIndex[s3a://qa.dssa.thetradedesk.com/data/datacamp/flights_csv/AA_DFW_2018.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Destination Airport)], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight Number:string,Destination Airport:string,Actual elapsed ti...
+- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, true]))
   +- *(2) Project [AIRPORTNAME#198, IATA#199]
      +- *(2) Filter isnotnull(IATA#199)
         +- *(2) FileScan parquet [AIRPORTNAME#198,IATA#199] Batched: 

## Comparing broadcast vs normal joins
You've created two types of joins, normal and broadcasted. Now your manager would like to know what the performance improvement is by using Spark optimizations. If the results are promising, you'll be given more opportunity to tweak the Spark setup as needed.

In [9]:
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = normal_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

Normal count:		119910	duration: 2.531486
Broadcast count:	119910	duration: 1.844353


While the difference in time is miniscule for our example, the ratio between the durations is significant. Depending on the makeup of the data being joined, you can notably cut the run time for Spark operations.